In [1]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from modules import extract_, compress_, load_
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('Plan_A').getOrCreate()

Exception: Java gateway process exited before sending its port number

In [3]:
# Plan_A : transfrom_flat 함수코드 수정 없이
# udf로 만들어 spark에서 의도대로 동작하는지 확인
# 함수는 flattened_data 를 반환

def transform_flat(date):
    '''
    날짜를 입력하면 그 날짜의 모든 거래를 품목별 법인별로 집계하여\n
    단일 json 데이터로 반환하는 함수
    '''
    import os
    import math
    import json
    import pandas as pd
    from dotenv import load_dotenv
    load_dotenv()

    api_id = os.getenv('garak_id')
    api_pw = os.getenv('garak_passwd')
    url = 'http://www.garak.co.kr/publicdata/dataOpen.do?'

    bubin_list = ['11000101','11000102','11000103','11000104','11000105','11000106']
    pummok_list = ['감귤','감자','건고추','고구마','단감','당근','딸기','마늘','무',
                    '미나리','바나나','배','배추','버섯','사과','상추','생고추','수박',
                    '시금치','양배추','양상추','양파','오이','참외','토마토','파',
                    '포도','피망','호박']

    dict1 = {'data': []}
    for pummok in pummok_list:
        dict2 = {f'{pummok}': []}
        for bubin in bubin_list:
            params = (
                    ('id', api_id),
                    ('passwd', api_pw),
                    ('dataid', 'data12'),
                    ('pagesize', '10'),
                    ('pageidx', '1'),
                    ('portal.templet', 'false'),
                    ('s_date', date),
                    ('s_bubin', bubin),
                    ('s_pummok', pummok),
                    ('s_sangi', '')
                    )
            dict3 = {f'{bubin}': []}
            list_total_count = int(extract_.extract(url, params)['lists']['list_total_count'])
            total_page = math.ceil(int(list_total_count) / 10)

            if int(list_total_count) != 0:
                for page in range(1, total_page+1):
                    params = (
                                ('id', api_id),
                                ('passwd', api_pw),
                                ('dataid', 'data12'),
                                ('pagesize', '10'),
                                ('pageidx', page),
                                ('portal.templet', 'false'),
                                ('s_date', date),
                                ('s_bubin', bubin),
                                ('s_pummok', pummok),
                                ('s_sangi', '')
                             )
                    html_dict = extract_.extract(url, params)
                    if list_total_count % 10 > 1:
                        for i in range(len(html_dict['lists']['list'])):
                            dict3[f'{bubin}'].append({
                                'idx' : ((page -1) * 10) + (i + 1),
                                'PUMMOK' : html_dict['lists']['list'][i]['PUMMOK'],
                                'PUMJONG' : html_dict['lists']['list'][i]['PUMJONG'],
                                'UUN' : html_dict['lists']['list'][i]['UUN'],
                                'DDD' : html_dict['lists']['list'][i]['DDD'],
                                'PPRICE' : html_dict['lists']['list'][i]['PPRICE'],
                                'SSANGI' : html_dict['lists']['list'][i]['SSANGI'],
                                'CORP_NM' : html_dict['lists']['list'][i]['CORP_NM'],
                                'ADJ_DT' : html_dict['lists']['list'][i]['ADJ_DT']
                                })
                    elif list_total_count % 10 == 1:
                        if list_total_count > 1:
                            for i in range(10):
                                dict3[f'{bubin}'].append({
                                    'idx' : ((page -1) * 10) + (i + 1),
                                    'PUMMOK' : html_dict['lists']['list'][i]['PUMMOK'],
                                    'PUMJONG' : html_dict['lists']['list'][i]['PUMJONG'],
                                    'UUN' : html_dict['lists']['list'][i]['UUN'],
                                    'DDD' : html_dict['lists']['list'][i]['DDD'],
                                    'PPRICE' : html_dict['lists']['list'][i]['PPRICE'],
                                    'SSANGI' : html_dict['lists']['list'][i]['SSANGI'],
                                    'CORP_NM' : html_dict['lists']['list'][i]['CORP_NM'],
                                    'ADJ_DT' : html_dict['lists']['list'][i]['ADJ_DT']
                                    })
                            list_total_count -= 10
                        elif list_total_count == 1:
                            dict3[f'{bubin}'].append({
                                'idx' : int(html_dict['lists']['list_total_count']),
                                'PUMMOK' : html_dict['lists']['list']['PUMMOK'],
                                'PUMJONG' : html_dict['lists']['list']['PUMJONG'],
                                'UUN' : html_dict['lists']['list']['UUN'],
                                'DDD' : html_dict['lists']['list']['DDD'],
                                'PPRICE' : html_dict['lists']['list']['PPRICE'],
                                'SSANGI' : html_dict['lists']['list']['SSANGI'],
                                'CORP_NM' : html_dict['lists']['list']['CORP_NM'],
                                'ADJ_DT' : html_dict['lists']['list']['ADJ_DT']
                                })
                dict2[f'{pummok}'].append(dict3)
            else:
                pass
        dict1['data'].append(dict2)

    flattened_data = []
    for item_data in dict1['data']:
        for item, bubin_list in item_data.items():
            for bubin_data in bubin_list:
                for bubin, transactions in bubin_data.items():
                    for transaction in transactions:
                        flattened_row = transaction.copy()
                        flattened_row['item'] = item
                        flattened_row['bubin'] = bubin
                        flattened_data.append(flattened_row)

    return flattened_data

In [4]:
schema = ArrayType(
    StructType([
        StructField("idx", IntegerType(), True),
        StructField("PUMMOK", StringType(), True),
        StructField("PUMJONG", StringType(), True),
        StructField("UUN", StringType(), True),
        StructField("DDD", StringType(), True),
        StructField("PPRICE", StringType(), True),
        StructField("SSANGI", StringType(), True),
        StructField("CORP_NM", StringType(), True),
        StructField("ADJ_DT", StringType(), True),
        StructField("item", StringType(), True),
        StructField("bubin", StringType(), True),
    ])
)

transform_flat_udf = udf(transform_flat, schema)

In [5]:
data = [("2023-04-01",), ("2023-04-02",), ("2023-04-03",)]
columns = ["date"]
input_df = spark.createDataFrame(data, columns)

In [6]:
input_df

DataFrame[date: string]

In [7]:
input_df.show()

Py4JJavaError: An error occurred while calling o40.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [ ]:
result_df = input_df.withColumn("flattened_data", transform_flat_udf(input_df["date"]))

In [8]:
spark.stop()